In [1]:
import copernicusmarine
import xarray as xr
import fsspec
import pandas as pd
from seapopym.standard.units import StandardUnitsLabels

## Load DATA


### Observations


In [2]:
bats_obs = xr.open_dataset("./products/Bats_obs.zarr", engine="zarr")
bats_obs

<xarray.Dataset> Size: 13kB
Dimensions:    (time: 538, latitude: 1, longitude: 1, layer: 1)
Coordinates:
  * latitude   (latitude) float64 8B 31.6
  * layer      (layer) int64 8B 0
  * longitude  (longitude) float64 8B -64.2
  * time       (time) datetime64[ns] 4kB 1995-05-10 1995-05-11 ... 2022-12-13
Data variables:
    day        (time, latitude, longitude, layer) float64 4kB ...
    night      (time, latitude, longitude, layer) float64 4kB ...
Attributes:
    creation date:  2024-10-13T20:03:40.209673
    layer:          epipelagic
    station:        Bats

In [3]:
LATITUDE = bats_obs.latitude.data[0]
LONGITUDE = bats_obs.longitude.data[0]
# The observation data time range is starting too early. We will use the time range of the model data.
# START_TIME = pd.Timestamp(bats_obs.time.data[0]).strftime("%Y-%m-%d")
# END_TIME = pd.Timestamp(bats_obs.time.data[-1]).strftime("%Y-%m-%d")
START_TIME = "1998-01-01"
END_TIME = "2022-12-31"
DELTA_IMPORT = 0.5
DELTA_COMPUTE = 0.1

print(
    f"Latitude: {LATITUDE}\nLongitude: {LONGITUDE}\nStart Time: {START_TIME}\nEnd Time: {END_TIME}\nDelta import: {DELTA_IMPORT}"
)

Latitude: 31.6
Longitude: -64.2
Start Time: 1998-01-01
End Time: 2022-12-31
Delta import: 0.5


### CMEMS


In [4]:
FROM_API = True
path_cmems_product_bio = "..."
path_cmems_product_physic = "..."

In [5]:
# copernicusmarine.login()

coordinates = {
    "minimum_latitude": LATITUDE - DELTA_IMPORT,
    "maximum_latitude": LATITUDE + DELTA_IMPORT,
    "minimum_longitude": LONGITUDE - DELTA_IMPORT,
    "maximum_longitude": LONGITUDE + DELTA_IMPORT,
    "start_datetime": START_TIME,
    "end_datetime": END_TIME,
}

if FROM_API:
    bio = copernicusmarine.open_dataset(dataset_id="cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i", **coordinates)
    bio.load()
    physic = copernicusmarine.open_dataset(dataset_id="cmems_mod_glo_bgc_my_0.083deg-lmtl-Fphy_PT1D-i", **coordinates)
    physic.load()
else:
    bio = xr.load_dataset(path_cmems_product_bio)
    physic = xr.load_dataset(path_cmems_product_physic)

INFO - 2025-01-15T10:33:07Z - Selected dataset version: "202411"
INFO - 2025-01-15T10:33:07Z - Selected dataset part: "default"
INFO - 2025-01-15T10:33:29Z - Selected dataset version: "202411"
INFO - 2025-01-15T10:33:29Z - Selected dataset part: "default"


In [6]:
with xr.set_options(keep_attrs=True):
    data = xr.merge([bio, physic])
    lat_attrs, lon_attrs = data.latitude.attrs, data.longitude.attrs
    data = data.sel(
        time=slice(START_TIME, END_TIME),
    ).sel(
        latitude=slice(LATITUDE - DELTA_COMPUTE, LATITUDE + DELTA_COMPUTE),
        longitude=slice(LONGITUDE - DELTA_COMPUTE, LONGITUDE + DELTA_COMPUTE),
    )
    data = data.mean(dim=["latitude", "longitude"], skipna=True)
    data = data.expand_dims({"latitude": [LATITUDE], "longitude": [LONGITUDE]})
    data.latitude.attrs, data.longitude.attrs = lat_attrs, lon_attrs
    data["T"].attrs["units"] = str(StandardUnitsLabels.temperature.units)
data

<xarray.Dataset> Size: 2MB
Dimensions:              (latitude: 1, longitude: 1, time: 9131, depth: 3)
Coordinates:
  * latitude             (latitude) float64 8B 31.6
  * longitude            (longitude) float64 8B -64.2
  * time                 (time) datetime64[ns] 73kB 1998-01-01 ... 2022-12-31
  * depth                (depth) int32 12B 1 2 3
Data variables: (12/13)
    mnkc_epi             (latitude, longitude, time) float64 73kB 0.7319 ... ...
    mnkc_hmlmeso         (latitude, longitude, time) float64 73kB 0.681 ... 0...
    mnkc_lmeso           (latitude, longitude, time) float64 73kB 1.118 ... 1...
    mnkc_mlmeso          (latitude, longitude, time) float64 73kB 0.5542 ... ...
    mnkc_mumeso          (latitude, longitude, time) float64 73kB 0.6062 ... ...
    mnkc_umeso           (latitude, longitude, time) float64 73kB 0.3349 ... ...
    ...                   ...
    zeu                  (latitude, longitude, time) float64 73kB 66.78 ... 7...
    zooc                 (latitude, longitude, time) float64 73kB 0.3745 ... ...
    T                    (latitude, longitude, time, depth) float64 219kB 20....
    U                    (latitude, longitude, time, depth) float64 219kB 0.0...
    V                    (latitude, longitude, time, depth) float64 219kB -0....
    pelagic_layer_depth  (latitude, longitude, time, depth) float64 219kB 100...
Attributes:
    Conventions:  CF-1.7
    references:   http://www.cls.fr; http://www.seapodym.eu
    institution:  CLS
    history:      Created on 2024-10-17
    source:       SEAPODYM-LMTL 3.0.0
    title:        Global ocean low and mid trophic levels biomass content hin...

## Change the depth to be categorical AND starting with 0 value rather than 1


In [7]:
depth_attrs = {
    "flag_values": "[0 1 2]",
    "flag_meanings": "epipelagic, upper-mesopelagic, lower-mesopelagic",
    "desc": "The layer depth according to Seapodym definition",
    "standard_name": "depth",
    "long_name": "Layer depth",
    "axis": "Z",
}
data = data.assign_coords(depth=[0, 1, 2])
data.depth.attrs = depth_attrs
data

<xarray.Dataset> Size: 2MB
Dimensions:              (latitude: 1, longitude: 1, time: 9131, depth: 3)
Coordinates:
  * latitude             (latitude) float64 8B 31.6
  * longitude            (longitude) float64 8B -64.2
  * time                 (time) datetime64[ns] 73kB 1998-01-01 ... 2022-12-31
  * depth                (depth) int64 24B 0 1 2
Data variables: (12/13)
    mnkc_epi             (latitude, longitude, time) float64 73kB 0.7319 ... ...
    mnkc_hmlmeso         (latitude, longitude, time) float64 73kB 0.681 ... 0...
    mnkc_lmeso           (latitude, longitude, time) float64 73kB 1.118 ... 1...
    mnkc_mlmeso          (latitude, longitude, time) float64 73kB 0.5542 ... ...
    mnkc_mumeso          (latitude, longitude, time) float64 73kB 0.6062 ... ...
    mnkc_umeso           (latitude, longitude, time) float64 73kB 0.3349 ... ...
    ...                   ...
    zeu                  (latitude, longitude, time) float64 73kB 66.78 ... 7...
    zooc                 (latitude, longitude, time) float64 73kB 0.3745 ... ...
    T                    (latitude, longitude, time, depth) float64 219kB 20....
    U                    (latitude, longitude, time, depth) float64 219kB 0.0...
    V                    (latitude, longitude, time, depth) float64 219kB -0....
    pelagic_layer_depth  (latitude, longitude, time, depth) float64 219kB 100...
Attributes:
    Conventions:  CF-1.7
    references:   http://www.cls.fr; http://www.seapodym.eu
    institution:  CLS
    history:      Created on 2024-10-17
    source:       SEAPODYM-LMTL 3.0.0
    title:        Global ocean low and mid trophic levels biomass content hin...

## Gathering Data & Save


In [8]:
data.to_zarr("./products/Bats_cmems.zarr", mode="w")